Давайте сформируем словарь параметров для получения информации о количестве визитов и просмотров страниц для разных источников трафика за период с начала лета 2019 года до вчерашнего дня включительно. В случае, если данных в отчёте много, предусмотрим загрузку первых 10 000 строк отчёта:

In [1]:
params = {'metrics': 'ym:s:visits,ym:s:pageviews',
          'dimensions': 'ym:s:referer',
          'date1': '2019-06-01',
          'date2': 'yesterday',
          'limit': 10000,
          'ids': 30177909}

#### Задание 1

Создайте словарь params, содержащий параметры запроса для определения количества уникальных посетителей (ym:s:users) из разных стран (ym:s:regionCountry) за период с 1 по 31 января 2019 года включительно. Не используйте в запросе параметры limit и offset, но при этом не забудьте задать правильный ID счётчика.

In [2]:
params = {'metrics': 'ym:s:users',
          'dimensions': 'ym:s:regionCountry',
          'date1': '2019-12-01',
          'date2': '2019-12-31',
          'ids': 30177909}

#### Задание 2
Создайте словарь params, содержащий параметры запроса для определения количества новых пользователей (ym:s:newUsers) с разбивкой их по дням недели, когда совершен визит (ym:s:dayOfWeek) и по типу устройства, с которого было посещение (ym:s:deviceCategory) за март 2019 года. Не используйте в запросе параметры limit и offset, но при этом не забудьте задать правильный ID счётчика.

In [ ]:
params={'metrics':'ym:s:newUsers',
       'dimensions':'ym:s:dayOfWeek,ym:s:deviceCategory',
       'date1':'2019-03-01',
       'date2':'2019-03-31',
       'ids':30177909}

#### Задание 3
Создайте словарь params, содержащий параметры запроса для определения средней продолжительности визита (ym:s:avgVisitDurationSeconds) и количества просмотренных страниц (ym:s:pageviews) в разные дни (ym:s:date) в зависимости от языка браузера (ym:s:browserLanguage) за прошедшие 5 дней (5daysAgo) и до сегодняшнего дня включительно. Не используйте в запросе параметры limit и offset, но при этом не забудьте задать правильный ID счётчика.

In [ ]:
params={'metrics':'ym:s:avgVisitDurationSeconds,ym:s:pageviews',
       'dimensions':'ym:s:date,ym:s:browserLanguage',
       'date1':'5daysAgo',
       'date2':'today',
       'ids':30177909}

## B10.3.2 Получение и анализ данных

In [3]:
import json 
import requests
from pprint import pprint

In [4]:
token = 'AgAAAAAUFRN5AAYOhtaStbXJX0exly6jAsVUGGY' # вместо слов "ваш_токен" нужно вставить свой токен
headers = {'Authorization': 'OAuth ' + token}

In [5]:
params = {'metrics': 'ym:s:visits,ym:s:pageviews', 
          'dimensions': 'ym:s:referer',  
          'date1': '2019-06-01',  
          'date2': '2019-06-10',     
          'ids': 30177909}

In [6]:
response = requests.get('https://api-metrika.yandex.net/stat/v1/data', params=params, headers=headers)
print (response.status_code)

200


In [7]:
metrika_data = response.json()

In [93]:
# metrika_data

In [14]:
print (metrika_data.keys())

dict_keys(['query', 'data', 'total_rows', 'total_rows_rounded', 'sampled', 'sample_share', 'sample_size', 'sample_space', 'data_lag', 'totals', 'min', 'max'])


In [18]:
# print (metrika_data["data"])

In [57]:
print(metrika_data['total_rows'])

112


In [58]:
metrika_data["data"][0]

{'dimensions': [{'name': 'https://google.com/', 'favicon': 'google.com'}],
 'metrics': [384.0, 444.0]}

#### Задание 1
Какое максимальное количество визитов из одного источника было зафиксировано в отчёте?

In [55]:
metrika_data["max"][0]

384.0

#### Задание 2
Из какого источника было совершено максимальное количество визитов на сайт в исследованный период времени?
- https://google.com 

In [61]:
metrika_data["data"][0]

{'dimensions': [{'name': 'https://google.com/', 'favicon': 'google.com'}],
 'metrics': [384.0, 444.0]}

#### Задание 3
Какие значения параметра 'favicon' НЕ встречаются в отчёте?

In [88]:
metrika_data['data'][0].values()

dict_values([[{'name': 'https://google.com/', 'favicon': 'google.com'}], [384.0, 444.0]])

In [94]:
# metrika_data['data']

#### Задание 4
Для скольких источников в рассмотренный период зафиксирован только один просмотр?

In [97]:
import pandas as pd
from collections import Counter
import json 
import requests
from pprint import pprint

token = 'AgAAAAAUFRN5AAYOhtaStbXJX0exly6jAsVUGGY'
headers = {'Authorization': 'OAuth' + token}
params = {'metrics': 'ym:s:visits', 
          'dimensions': 'ym:s:lastSearchPhrase',  
          'date1': '2019-05-01',  
          'date2': '2019-05-31',
          'limit': 10000,
          'ids': 30177909}
response = requests.get('https://api-metrika.yandex.net/stat/v1/data', params=params, headers=headers)
metrika_data = response.json()
result = []
# Начинаем перебор элементов старого отчёта (только раздел с данными)
for data_item in metrika_data['data']:
    # Создаём словарь для хранения текущих данных в новом формате 
    new_dict = {}  
    # Обращаемся к разделу отчёта, содержащего метаданные
    # Перебираем названия группировок, которые использовались в отчёте
    for i,dimension in enumerate(data_item['dimensions']):
        # Создаём в новом словаре ключ для каждой группировки
        # Устанавливаем значение для каждого ключа
        new_dict[metrika_data['query']['dimensions'][i]] = dimension['name']
    # Те же действия выполняем для метрик
    for i,metric in enumerate(data_item['metrics']):
        new_dict[metrika_data['query']['metrics'][i]] = metric
    # Добавляем созданный словарь в итоговый список result
    result.append(new_dict)
y=0
for x in result:
    if 'python' in x['ym:s:lastSearchPhrase'].lower():
        y+=x['ym:s:visits']
print(y)

68.0


In [98]:
import json
import requests
from pprint import pprint
import pandas as pd
token = 'AgAAAAAUFRN5AAYOhtaStbXJX0exly6jAsVUGGY'
headers = {'Authorization': 'OAuth ' + token}

params = {'metrics': 'ym:s:visits,ym:s:pageviews',
          'dimensions': 'ym:s:referer',
          'date1': '2019-06-01',
          'date2': '2019-06-10',
          'ids': 30177909}

response = requests.get('https://api-metrika.yandex.net/stat/v1/data', params=params, headers=headers)
aaa = response.json()['data']
result_list=[]

for i in aaa:
    result_list.append(i['metrics'][1])

print(pd.Series(result_list).value_counts())

1.0      64
2.0      19
5.0       4
3.0       3
4.0       3
7.0       1
11.0      1
13.0      1
10.0      1
65.0      1
146.0     1
444.0     1
dtype: int64


#### B10.3.3 Работа со структурой отчёта

In [101]:
token = 'AgAAAAAUFRN5AAYOhtaStbXJX0exly6jAsVUGGY'
headers = {'Authorization': 'OAuth' + token}
params = {'metrics': 'ym:s:visits', 
          'dimensions': 'ym:s:lastSearchPhrase',  
          'date1': '2019-05-01',  
          'date2': '2019-05-31',
          'limit': 10000,
          'ids': 30177909}

response = requests.get('https://api-metrika.yandex.net/stat/v1/data', params=params, headers=headers)
metrika_data = response.json()
result = []

# Начинаем перебор элементов старого отчёта (только раздел с данными)
for data_item in metrika_data['data']:
    # Создаём словарь для хранения текущих данных в новом формате 
    
    new_dict = {}  
    # Обращаемся к разделу отчёта, содержащего метаданные
    # Перебираем названия группировок, которые использовались в отчёте
    
    for i,dimension in enumerate(data_item['dimensions']):
        # Создаём в новом словаре ключ для каждой группировки
        # Устанавливаем значение для каждого ключа
        
        new_dict[metrika_data['query']['dimensions'][i]] = dimension['name']
    # Те же действия выполняем для метрик
    
    for i,metric in enumerate(data_item['metrics']):
        new_dict[metrika_data['query']['metrics'][i]] = metric
    # Добавляем созданный словарь в итоговый список result
    
    result.append(new_dict)
# result

#### Задание 1
Посчитайте среднесуточное количество пользователей за май 2019 года. Для того чтобы выгрузить данные по дням, используйте группировку ym:s:date, для получения количества пользователей используйте метрику ym:s:users. В ответе укажите число с округлением до 2 знаков после запятой.

In [115]:
token = 'AgAAAAAUFRN5AAYOhtaStbXJX0exly6jAsVUGGY'
headers = {'Authorization': 'OAuth' + token}
params = {'metrics': 'ym:s:users', 
          'dimensions': 'ym:s:date',  
          'date1': '2019-05-01',  
          'date2': '2019-05-31',
          'limit': 10000,
          'ids': 30177909}

response = requests.get('https://api-metrika.yandex.net/stat/v1/data', params=params, headers=headers)
data = response.json()
data

{'query': {'ids': [30177909],
  'dimensions': ['ym:s:date'],
  'metrics': ['ym:s:users'],
  'sort': ['-ym:s:users'],
  'date1': '2019-05-01',
  'date2': '2019-05-31',
  'limit': 10000,
  'offset': 1,
  'group': 'Week',
  'auto_group_size': '1',
  'quantile': '50',
  'offline_window': '21',
  'attribution': 'Last',
  'currency': 'RUB',
  'adfox_event_id': '0'},
 'data': [{'dimensions': [{'name': '2019-05-16'}], 'metrics': [124.0]},
  {'dimensions': [{'name': '2019-05-13'}], 'metrics': [117.0]},
  {'dimensions': [{'name': '2019-05-20'}], 'metrics': [115.0]},
  {'dimensions': [{'name': '2019-05-22'}], 'metrics': [115.0]},
  {'dimensions': [{'name': '2019-05-15'}], 'metrics': [105.0]},
  {'dimensions': [{'name': '2019-05-14'}], 'metrics': [103.0]},
  {'dimensions': [{'name': '2019-05-23'}], 'metrics': [103.0]},
  {'dimensions': [{'name': '2019-05-29'}], 'metrics': [100.0]},
  {'dimensions': [{'name': '2019-05-30'}], 'metrics': [100.0]},
  {'dimensions': [{'name': '2019-05-21'}], 'metrics':

#### B10.3.3 Работа со структурой отчёта Задание 3
Посчитайте, на сколько процентов уменьшилось число посещений (ym:s:visits) в мае 2019 относительно апреля 2019,
где страница входа (группировка ym:s:startURL) соответствует странице
'http://datalytics.ru/all/stanovyas-guru-api-yandeks-metriki/'. Месяц можно получить,
используя группировку ym:s:month. Ответ дайте целых числом (без указания знака "минус")

In [104]:
import requests
from pprint import pprint
import json

headers = {'Authorization': 'OAuth ' + token}
params ={
   'metrics': 'ym:s:visits',  # Показатели: Посещения
   'dimensions': 'ym:s:month,ym:s:startURL',  # Параметры: Ключевая фраза
   'date1': '2019-05-01',  # Дата старта
   'date2': '2019-05-31',  # Дата конца
   'limit': 100000,  # Количество строк
   'ids': 30177909  # ID счетчика
}

response = requests.get('https://api-metrika.yandex.net/stat/v1/data', params=params, headers=headers)
metrika_data= response.json()
#print(metrika_data.keys())

print(metrika_data['data'][2]['metrics'][0]) # 291.0

num_05 = metrika_data['data'][2]['metrics'][0]
headers = {'Authorization': 'OAuth ' + token}
params ={
   'metrics': 'ym:s:visits',  # Показатели: Посещения
   'dimensions': 'ym:s:month,ym:s:startURL',  # Параметры: Ключевая фраза
   'date1': '2019-04-01',  # Дата старта
   'date2': '2019-04-30',  # Дата конца
   'limit': 100000,  # Количество строк
   'ids': 30177909  # ID счетчика
}
response = requests.get('https://api-metrika.yandex.net/stat/v1/data', params=params, headers=headers)
metrika_data= response.json()
#print(metrika_data.keys())
print(metrika_data['data'][2]['metrics'][0]) # 384.0

num_04 = metrika_data['data'][2]['metrics'][0]
print('Ответ B10.3.3 Работа со структурой отчёта Задание 3 =',100-(1-(num_04-num_05)/num_04)*100) # Ответ 24.22

291.0
384.0
Ответ B10.3.3 Работа со структурой отчёта Задание 3 = 24.21875


#### Задание 2
Проанализируйте сумму посещений (ym:s:visits) по ключевым словам (группировка ym:s:lastSearchPhrase) за май 2019 года. Сколько посещений содержит в себе подстроку 'python'? Учитывайте, что в ключевых фразах Python может писаться с большой буквы, но при подсчёте надо игнорировать регистр символов

In [107]:
params = {
   'metrics': 'ym:s:visits', # Показатели: Посещения
   'dimensions': 'ym:s:lastSearchPhrase', # Параметры: Ключевая фраза
   'date1': '2019-05-01', # Дата старта
   'date2': '2019-05-31', # Дата конца
   'limit': 100000, # Количество строк
   'ids': 30177909 # ID счетчика
}

# response = requests.get('https://api-metrika.yandex.net/stat/v1/data', params=params, headers=headers)
# data_2 = response.json()

count = 0
for n in result:
    if 'python' in n['ym:s:lastSearchPhrase'].lower():
        count += n['ym:s:visits']

print(count)

68.0


## B10.4.2 Итоговый тест
#### Задание 1

Отметьте, в каких целях можно использовать API Яндекс.Метрики.
- для использования данных Яндекс.Метрики в собственных веб-сервисах и алгоритмах

#### Задание 2
Каким образом Яндекс.Метрика собирает данные о посещаемости веб-сайта?
- на страницах сайта устанавливается специальный код отслеживания, который передаёт в Яндекс.Метрику данные о посещении сайт

#### Задание 3
В Яндекс.Метрике численные величины, которые отражают сводные показатели визитов или действий — это
- метрики

#### Задание 4
Что такое группировка в терминологии Яндекс.Метрики?
- какой-либо атрибут визита или действия, по которому объединяются данные 

#### Задание 5
Как называется HTTP-заголовок, в котором передается авторизационный токен Яндекс.Метрики?
- Authorization

#### Задание 6
Какой параметр запроса к API Яндекс.Метрики позволяет задать список группировок?
- dimensions

#### Задание 7
Что позволяет задать параметр offset?
- индекс первой строки выборки

#### Задание 8
Какая группировка позволяет вывести в отчёте адрес страницы входа (посадочной страницы)?
- ym:s:startURL

#### Задание 9
В каком формате возвращает данные API Яндекс.Метрики?
- JSON

#### Задание 10
Какой ключ в отчёте, полученном от API Яндекс.Метрики, позволяет вывести параметры исходного запроса?
- query